# SME0829 Aprendizado de Máquina


por **Cibele Russo**

**ICMC/USP - São Carlos SP**



## Implementação CART - Árvore de classificação



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

random.seed(1)

# Carregando os dados
dados = pd.read_csv('https://raw.githubusercontent.com/cibelerusso/Aprendizado_de_Maquina/main/Dados/JogarTenis.csv')
dados.head(14)


,Tempo,Temperatura,Umidade,Vento,Joga
0,Chuvoso,22,91,Sim,Não
1,Ensolarado,21,70,Não,Sim
2,Ensolarado,27,90,Sim,Não
3,Nublado,28,86,Não,Sim
4,Chuvoso,21,96,Não,Sim
5,Chuvoso,18,70,Sim,Não
6,Nublado,18,65,Sim,Sim
7,Nublado,22,90,Sim,Sim
8,Ensolarado,24,70,Sim,Sim
9,Chuvoso,20,80,Não,Sim


In [2]:
# Converte as colunas categóricas em números
converte = {'Ensolarado': 1, 'Nublado': 2, 'Chuvoso': 3, 'Sim': 1, 'Não': 0}
dados = dados.applymap(lambda s: converte.get(s) if s in converte else s)
dados

,Tempo,Temperatura,Umidade,Vento,Joga
0,3,22,91,1,0
1,1,21,70,0,1
2,1,27,90,1,0
3,2,28,86,0,1
4,3,21,96,0,1
5,3,18,70,1,0
6,2,18,65,1,1
7,2,22,90,1,1
8,1,24,70,1,1
9,3,20,80,0,1


In [3]:
# Separa as preditoras (X) e a resposta (y)
X = dados.drop('Joga', axis=1)
y = dados['Joga']

In [4]:
# Implementação de uma árvore de classificação

class DecisionTreeNode:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

# Função para Índice de Gini como impureza
def gini_impurity(data):
    total_samples = len(data)
    if total_samples == 0:
        return 0    
    num_positive = sum(data[:, -1])
    p_positive = num_positive / total_samples
    p_negative = 1 - p_positive    
    impurity = 1 - p_positive**2 - p_negative**2
    return impurity

# Função para dividir os dados com base em uma variável e ponto de corte
def split_data(data, feature, threshold):
    left = data[data[:, feature] <= threshold]
    right = data[data[:, feature] > threshold]
    return left, right

# Função para encontrar a melhor divisão
def find_best_split(data):
    features = data.shape[1] - 1
    num_samples = len(data)
    best_gini = 1.0
    best_feature = None
    best_threshold = None    
    for feature in range(features):
        thresholds = np.unique(data[:, feature])
        for threshold in thresholds:
            left, right = split_data(data, feature, threshold)            
            if len(left) == 0 or len(right) == 0:
                continue            
            gini_left = gini_impurity(left)
            gini_right = gini_impurity(right)
            gini = (len(left) / num_samples) * gini_left + (len(right) / num_samples) * gini_right
            if gini < best_gini:
                best_gini = gini
                best_feature = feature
                best_threshold = threshold
    return best_feature, best_threshold

# Função para construir a árvore
def build_tree(data, depth=0, max_depth=None):
    num_samples, num_features = data.shape
    labels = data[:, -1]
    # Critérios de parada
    if len(np.unique(labels)) <= 1 or depth == max_depth:
        leaf_value = np.argmax(np.bincount(labels))
        return DecisionTreeNode(value=leaf_value)
    best_feature, best_threshold = find_best_split(data)
    left_data, right_data = split_data(data, best_feature, best_threshold)
    
    left_subtree = build_tree(left_data, depth+1, max_depth)
    right_subtree = build_tree(right_data, depth+1, max_depth)
    
    return DecisionTreeNode(feature=best_feature, threshold=best_threshold, left=left_subtree, right=right_subtree)

# Previsões usando a árvore de classificação
def predict(tree, X):
    if tree.value is not None:
        return tree.value
    if X[tree.feature] <= tree.threshold:
        return predict(tree.left, X)
    else:
        return predict(tree.right, X)


In [5]:
data = np.array(dados)
data

array([[ 3, 22, 91,  1,  0],
       [ 1, 21, 70,  0,  1],
       [ 1, 27, 90,  1,  0],
       [ 2, 28, 86,  0,  1],
       [ 3, 21, 96,  0,  1],
       [ 3, 18, 70,  1,  0],
       [ 2, 18, 65,  1,  1],
       [ 2, 22, 90,  1,  1],
       [ 1, 24, 70,  1,  1],
       [ 3, 20, 80,  0,  1],
       [ 2, 27, 75,  0,  1],
       [ 1, 29, 85,  0,  0],
       [ 1, 22, 95,  0,  0],
       [ 3, 24, 80,  0,  1]])

In [6]:
# Constrói a árvore
decision_tree = build_tree(data)


In [7]:
# Exemplo: Predizer se num dia chuvoso, temperatura 20 graus, umidade 90, com vento
new_data_point = np.array([3, 20, 90, 1])
prediction = predict(decision_tree, new_data_point)
if prediction == 0:
    print("Não joga")
else:
    print("Joga")

Joga
